# Part 00: Notebook overview ..

In [ ]:

#  This Notebook is part of a set that demonstrate GNN using a movie dataset.
#  About this Notebook,
#
#  .  In NoteBook 20*, we built a Movie graph; movies, keywords
#     In NoteBook 30* we delivered a primer on CountVectorizers, and Classifiers using non-graph.
#
#  .  Here we'll apply the non-graph Classifier to our Movie nodes from our graph.
#
#     We'll apply it to just Movie.title, then Movie.tagline, then Movie.overview.
#     And then we'll apply it to all 3 at one time.
#
#     And we'll compare the results to known Movie.genres_primary.
#
#        Recall Movies actually had an array of genres, and we derived genres_primary
#        as the genres in the first position inside the array.
#
#  .  Our graph, our source of data, was prepared/loaded in NoteBook 20*.



#  Part 01: Graph setup, and initial read

In [ ]:

#  Setting display options, and a flag for outputting more information
#

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

from tabulate import tabulate


# MY_DEBUG = True
MY_DEBUG = False
   #
print("--")


In [ ]:

#  The KatanaGraph remote API is expected to run from a node external to
#  the Katana Graph cluster itself.
#
#  This differs from the distributed API, which is meant to run primitives
#  on the Katana Graph worker nodes.
#

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph1" 

print("--")


In [ ]:

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


In [ ]:

#  Sanity check 
#
display("Number of Graph Nodes: %d" % (my_graph.num_nodes()))
display("Number of Graph Edges: %s" % (my_graph.num_edges()))


#  Sample output,
#
#     'Number of Graph Nodes: 64856'
#     'Number of Graph Edges: 330988'


In [ ]:

#  Looking at the graph
#
if (MY_DEBUG):
   l_result = my_graph.query("""
   
      MATCH (n) -[r]-> (m)
      RETURN n, r, m
      LIMIT 1000                        //  Limit is 25,000 for visualization, smaller is better
      
      """,
      contextualize=True)
   
   l_result.view()

print("--")



<div> 
<img src="./01_Images/10-Movie-Query-1.png" alt="Drawing" style="width: 1600px;"/>
</div>


#  Part 02: Getting data for use by our non-graph classifier

In [ ]:

#  We're using a non-graph set of libraries from sklearn. As such, we need to pull
#  the data out of the graph into DataFrames


In [ ]:

#  We'll consider these packages to be common knowledge. Else, return to the Compulsaries
#  set of NoteBooks for sample use, introduction ..
#
import numpy as np
import pandas as pd
   #
import dask.array as da
import dask.dataframe as dd
   #
from dask.dataframe import from_pandas

   ###
    
from sklearn.feature_extraction.text import CountVectorizer            #  Build a sparse vector (matrix) of keywords
from sklearn.feature_extraction.text import TfidfTransformer           #  Tool to norrmalize a condition; words that occur frequently, versus giving more weight to infrequent words
   #                                                                   #        Ie., Having 6 toes is way more rare than having 5. Should I give more weight that a person is 5'10", or has 6 toes ?
from sklearn.naive_bayes import MultinomialNB                          #  A non-graph ML routine, a Classifier  (similar perhaps to node property prediction, but on non-graph)
                                                                       #     Several variants of NaiveBayes, one being 'multi-nomial'.
                                                                       #     'multi-nomial', used commonly for word count style problems
        
from sklearn.pipeline import Pipeline                                  #  Allow us to simplify exuection of many sequential steps

print("--")


In [ ]:

#  These will be our test data set movies
#
#     (Previously we [ imported ] movie.id as a string, not an integer; no reason for not doing that.)
#

l_movies_test_ids = [
   "8469"    ,      #  Animal House
   "11848"   ,      #  Animal Farm
   "600"     ,      #  Full Metal Jacket
   "13342"   ,      #  Fast Times at Ridgemont High
   "10373"   ,      #  Quadrophenia
   "62"      ,      #  2001: A Space Odyssey
   "14328"   ,      #  The Paper Chase                            <-- Fyi, labeled as a Comedy and a Drama, with comedy in the first position
   "11589"   ,      #  Kelly's Heroes
   "694"     ,      #  The Shining
   "424"     ,      #  Schindler's List
   ]

l_query  = """
   MATCH (n: Movies) 
   WHERE n.id IN {0}
   RETURN n.id AS id, n.title AS title, n.genres_primary AS genres, n.genres_primary_id AS genres_id, n.tagline AS tagline, n.overview AS overview
   ORDER BY n.title
   """.format(l_movies_test_ids)

df_movies_test = my_graph.query(l_query)


if (MY_DEBUG):
   print(tabulate(df_movies_test, headers='keys', tablefmt='psql'))
   #
   #  Alternate to the above
   #
   #  display(df_movies_test.table())

   #  Displaying programmatically
   #
   print("")
   print("")
   for l_each in df_movies_test.itertuples():
      print("%8s   %-32s   %-64s" % (l_each.id, l_each.title, l_each.tagline))


print("--")

#  Sample output,
#
#     +----+-------+------------------------------+-----------------+-------------+------------------------------------------------------------------------
#     |    |    id | title                        | genres          |   genres_id | tagline                                                       
#     |----+-------+------------------------------+-----------------+-------------+----------------------------------------------------------------
#     |  0 |    62 | 2001: A Space Odyssey        | Science Fiction |         878 | An epic drama of adventure and exploration                           
#     |  1 | 11848 | Animal Farm                  | Animation       |          16 | He's got the world in an UPROAR!                                
#     |  2 |  8469 | Animal House                 | Comedy          |          35 | It was the Deltas against the rules... the rules lost!      
#     |  3 | 13342 | Fast Times at Ridgemont High | Comedy          |          35 | Fast Cars, Fast Girls, Fast Carrots...Fast Carrots?        
#     |  4 |   600 | Full Metal Jacket            | Drama           |          18 | Vietnam can kill me, but it can’t make me care.           
#     |  5 | 11589 | Kelly's Heroes               | Adventure       |          12 | They set out to rob a bank... and damn near won a war ins
#     |  6 | 10373 | Quadrophenia                 | Drama           |          18 | A Way of Life                                               
#     |  7 |   424 | Schindler's List             | Drama           |          18 | Whoever saves one life, saves the world entire.              
#     |  8 | 14328 | The Paper Chase              | Comedy          |          35 | You have to choose between the girl you love and the diploma you've worked for all
#     |  9 |   694 | The Shining                  | Horror          |          27 | A masterpiece of modern horror.                                          
#     +----+-------+------------------------------+-----------------+-------------+--------------------------------------------------------------------------
#     
#     
#           62   2001: A Space Odyssey              An epic drama of adventure and exploration                      
#        11848   Animal Farm                        He's got the world in an UPROAR!                                
#         8469   Animal House                       It was the Deltas against the rules... the rules lost!          
#        13342   Fast Times at Ridgemont High       Fast Cars, Fast Girls, Fast Carrots...Fast Carrots?             
#          600   Full Metal Jacket                  Vietnam can kill me, but it can’t make me care.                 
#        11589   Kelly's Heroes                     They set out to rob a bank... and damn near won a war instead!  
#        10373   Quadrophenia                       A Way of Life                                                   
#          424   Schindler's List                   Whoever saves one life, saves the world entire.                 
#        14328   The Paper Chase                    You have to choose between the girl you love and the diploma you've worked for all your life. You have 30 seconds.
#          694   The Shining                        A masterpiece of modern horror.
    


In [ ]:

%%time
#
#  Report time of execution for the cell:  takes just over 5 minutes for 60K rows.


#  Go get our train data set movies
#

#  We'll use this next DataFrame a lot moving forward.
#
#     .  For the first time we are returning more than a trivial number of rows.
#        As such, we use a different technique here to fetch.
#
#     .  Many straight up ML routines are not parallel ready, so we use a Pandas
#        DataFrame.
#

l_query  = """
   MATCH (n: Movies) 
   WHERE NOT n.id IN {0}
   RETURN n.id AS id, n.title AS title, n.genres_primary AS genres, n.genres_primary_id AS genres_id, n.tagline AS tagline, n.overview AS overview
   LIMIT 5000
   """.format(l_movies_test_ids)
      #
df_result = my_graph.query_paginated(l_query)


l_columns = ["id", "title", "genres", "genres_id", "tagline", "overview"]
   #
df_movies_train = pd.DataFrame()
   #
l_cntr1 = 0
l_cntr2 = 0
   #
for l_page in df_result:
   l_cntr1 += 1
   for l_each in l_page.itertuples():
      l_cntr2 += 1
         #
      l_row  = pd.DataFrame( np.array([[l_each.id, l_each.title, l_each.genres, l_each.genres_id, l_each.tagline, l_each.overview]], dtype="str"), columns = l_columns )
         #
      df_movies_train = df_movies_train.append(l_row)


print("Number of movies in DataFrame: %d" % (len(df_movies_train)))
print("")

if (MY_DEBUG):
   l_cntr = 0
      #
   for l_each in df_movies_train.itertuples():
      l_cntr += 1
         #
      if (l_cntr < 10):
         print("%8s   %-32s   %-64s" % (l_each.id, l_each.title, l_each.tagline))

    
print("--")

#  Recall, 64856 is count of [ all ] nodes

#  Sample output,
#
#     Number of movies in DataFrame: 45423
#     
#        54300   Live Forever                       The rise and fall of Brit Pop.                                  
#        75733   Perceval                           Unknown                                                         
#        82100   Flower Girl                        Unknown                                                         
#        58925   Mr. Klein                          Unknown                                                         
#         2502   The Bourne Supremacy               They should have left him alone.                                
#          314   Catwoman                           CATch her in IMAX                                               
#       389882   The Love-stricken                  Unknown                                                         
#        25451   Taking Chances                     Unknown                                                         
#        74012   Crash Dive                         Unknown 


In [50]:

#  Some of the values we received from the traversals may be empty strings. 
#  Check those.

if (MY_DEBUG):
   for l_each in ["id", "title", "tagline", "overview"]:
      print("Checking  %-18s   ...   total rows, %-10d   num missing/null, %-10d   value == 'Unknown', %-10d" % (
         l_each,
         len(df_movies_train),
         df_movies_train[l_each].isnull().sum(),
         df_movies_train[l_each].eq("Unknown").sum(),
         ))

print("--")

#  Sample output,
#
#     Checking  id                   ...   total rows, 45423        num missing/null, 0            value == 'unknown', 0         
#     Checking  title                ...   total rows, 45423        num missing/null, 0            value == 'unknown', 5         
#     Checking  tagline              ...   total rows, 45423        num missing/null, 0            value == 'unknown', 25032     
#     Checking  overview             ...   total rows, 45423        num missing/null, 0            value == 'unknown', 954  


Checking  id                   ...   total rows, 45423        num missing/null, 0            value == 'unknown', 0         
Checking  title                ...   total rows, 45423        num missing/null, 0            value == 'unknown', 5         
Checking  tagline              ...   total rows, 45423        num missing/null, 0            value == 'unknown', 25032     
Checking  overview             ...   total rows, 45423        num missing/null, 0            value == 'unknown', 954       
--


# Part 03:  Do the training

In [142]:

#  Two functions and a pipeline we will use many times below- 
#
#     Reminder:  the steps below were detailed at length in NoteBook 30*


#  Build a single structure with all values we need,
#
#     i_arg1    ::  a Pandas dataframe
#     i_arg2    ::  the named column to treat as data for the classifier
#     i_arg3    ::  the named column that is the class/type
#
def f_shape(i_arg1, i_arg2, i_arg3):
    
   my_dictionary = {}
      #
   my_dictionary["title" ] = i_arg1.title  .to_list()         #  Used in reporting only
      #
   my_dictionary["data"  ] = i_arg1[i_arg2].to_list()
   my_dictionary["class" ] = i_arg1[i_arg3].to_list()
    
   return my_dictionary


#  Define a pipeline
#
my_pipeline = Pipeline([
   ("cv",     CountVectorizer()  ),
   ("tfidf",  TfidfTransformer() ),
   ("clf",    MultinomialNB()    ),
   ])

print("--")


--


In [143]:

#  A function to output results,
#
#  i_arg1     ::  The test data set
#  i_arg2     ::  Results from the classifier
#
def f_print_results(i_arg1, i_arg2):
    
   display("Average accuracy: %f" % (np.mean(i_arg2 == i_arg1["class"])) )
   print("")
      #
   for l_index, l_zip in enumerate(zip(i_arg1["data"], i_arg1["class"])):
      print("Data: %-32s -- %-90s   Correct Class: %-18s   Predict Class: %-18s" % (my_test["title"][l_index], l_zip[0][0:90], l_zip[1], i_arg2[l_index]) )
   
   print("")
   print("--")
   print("")
    
print("--")


--


In [134]:
%%time

#  Score the model for test movies and output results
#

my_train = f_shape(df_movies_train, "title", "genres")
   #   
my_classifier = my_pipeline.fit(my_train["data"], my_train["class"])

    
my_test  = f_shape(df_movies_test , "title", "genres")
   #
my_result = my_classifier.predict(my_test["data"])
    
    
f_output_results(my_test, my_result)


#  **Note:  There are a couple of classifications in the source data we wouldn't agree with

#  Sample output,
#


'Average accuracy: 0.600000'


Data: 2001: A Space Odyssey            -- 2001: A Space Odyssey                                                                        Correct Class: Science Fiction      Predict Class: Comedy            
Data: Animal Farm                      -- Animal Farm                                                                                  Correct Class: Animation            Predict Class: Comedy            
Data: Animal House                     -- Animal House                                                                                 Correct Class: Comedy               Predict Class: Comedy            
Data: Fast Times at Ridgemont High     -- Fast Times at Ridgemont High                                                                 Correct Class: Comedy               Predict Class: Comedy            
Data: Full Metal Jacket                -- Full Metal Jacket                                                                            Correct Class: Drama                Predict 

In [140]:

#  The above was on movies.title. Repeat now for movies.tagline and movies.overview
#

my_train = f_shape(df_movies_train, "tagline", "genres")
   #   
my_classifier = my_pipeline.fit(my_train["data"], my_train["class"])

my_test  = f_shape(df_movies_test , "tagline", "genres")
   #
my_result = my_classifier.predict(my_test["data"])
    
f_output_results(my_test, my_result)


print("")
print("")


#  my_train = f_shape(df_movies_train, "overview", "genres")
#     #   
#  my_classifier = my_pipeline.fit(my_train["data"], my_train["class"])
#  
#  my_test  = f_shape(df_movies_test , "overview", "genres")
#     #
#  my_result = my_classifier.predict(my_test["data"])
#      
#  f_output_results(my_test, my_result)


print("")
print("")
print("--")


'Average accuracy: 0.300000'


Data: 2001: A Space Odyssey            -- An epic drama of adventure and exploration                                                   Correct Class: Science Fiction      Predict Class: Drama             
Data: Animal Farm                      -- He's got the world in an UPROAR!                                                             Correct Class: Animation            Predict Class: Comedy            
Data: Animal House                     -- It was the Deltas against the rules... the rules lost!                                       Correct Class: Comedy               Predict Class: Drama             
Data: Fast Times at Ridgemont High     -- Fast Cars, Fast Girls, Fast Carrots...Fast Carrots?                                          Correct Class: Comedy               Predict Class: Drama             
Data: Full Metal Jacket                -- Vietnam can kill me, but it can’t make me care.                                              Correct Class: Drama                Predict 

In [ ]:

my_test = {}
   #
my_test["data"] = [ 
   "I hate baseball" ,
   "I like wine"     ,
   "eggs with cheese",
   "eggs with cheese football",
   "football with eggs and cheese",
   "Wisconsin, America's Dairyland",
   ]


#  Applying/scoring the model
#
my_result = my_classifier.predict(my_test["data"])



#  Recall that the sorted class array is,
#
#     my_train["class"] = [ "food" , "sport", ]


my_test["target"] = [1, 0, 0, 0, 0, 0]                                     #  We only need this to gauge accuracy below, and the last one is wrong



display("Average accuracy: %f" % (np.mean(my_result == my_test["target"])) )
   #
for l_index, l_zip in enumerate(zip(my_test["data"], my_test["target"])):
   print("Data: %-36s   Correct Class: %-10s   Predict Class: %-10s" % (l_zip[0], my_train["class"][l_zip[1]], my_train["class"][my_result[l_index]]) )


#  Final piece, scoring (applying)

In [ ]:

#  Now we are ready to score (apply) the model.
#


#  Here is the test data
#
my_test = {}
   #
my_test["data"] = [ 
   "I hate baseball" ,
   "I like wine"     ,
   "eggs with cheese",
   "eggs with cheese football",
   "football with eggs and cheese",
   "Wisconsin, America's Dairyland",
   ]


#  Applying/scoring the model
#
my_result = my_classifier.predict(my_test["data"])



#  Recall that the sorted class array is,
#
#     my_train["class"] = [ "food" , "sport", ]


my_test["target"] = [1, 0, 0, 0, 0, 0]                                     #  We only need this to gauge accuracy below, and the last one is wrong



display("Average accuracy: %f" % (np.mean(my_result == my_test["target"])) )
   #
for l_index, l_zip in enumerate(zip(my_test["data"], my_test["target"])):
   print("Data: %-36s   Correct Class: %-10s   Predict Class: %-10s" % (l_zip[0], my_train["class"][l_zip[1]], my_train["class"][my_result[l_index]]) )

#  Sample output,
#
#     'Average accuracy: 0.833333'
#
#     Data: I hate baseball                        Correct Class: sport        Predict Class: sport     
#     Data: I like wine                            Correct Class: food         Predict Class: food      
#     Data: eggs with cheese                       Correct Class: food         Predict Class: food      
#     Data: eggs with cheese football              Correct Class: food         Predict Class: food      
#     Data: football with eggs and cheese          Correct Class: food         Predict Class: food      
#     Data: Wisconsin, America's Dairyland         Correct Class: food         Predict Class: sport     

